# Generate Autopology based inputs and data for a condensed phase system 

Currently, the functionality of incoporating bond priors is very limited you can compute bond, angle, and dihedral terms, but you need to make sure that you are using unwrapped cooridinates 

In [14]:
import sys
sys.path.append("../..")
sys.path.append("..")
import sys
import numpy as np
from importlib import reload
import networkx as nx
import torch
from torch.utils.data import DataLoader

import sys
sys.path.append("../..")

In [15]:
from tutorials.data.htvs_snippet import get_mol_ref
import nff.data as d
from nff.data import Dataset, split_train_validation_test, collate_dicts, sparsify_tensor
import pickle

In [16]:
# load ethane data
props = pickle.load( open( "./data/ethane_data.pkl", "rb" ) )
props['offsets'] = [sparsify_tensor(offset.matmul(torch.Tensor(props["cell"][i]))) for i, offset in enumerate(props['offsets'])]
props['smiles'] = ['CC'] * len(props['nxyz'])
dataset = d.Dataset(props.copy(), units='kcal/mol')


In [17]:
# construct bond list 
bond_dic = {'CC': [[2 * i, 2 * i + 1] for i in range(64)]}

In [18]:
# generate topologies
dataset.generate_topologies(bond_dic)

In [19]:

# number of features in the autopology feature vector
n_autopology_features = 256

autopology_params = {
    "n_features": n_autopology_features,
    "n_convolutions": 4,
    "conv_type": "double_node", # this type of convolution is a concatenation of features from
                                # both the node and the nodes it's bonded to. Another option is
                                # single_node, which doesn't use a concatenation
    "conv_update_layers": [{'name': 'linear', 'param' : {'in_features': int(2*n_autopology_features),
                                                    'out_features': n_autopology_features}},
                      {'name': 'Tanh', 'param': {}},
                      {'name': 'linear', 'param' : {'in_features': n_autopology_features,
                                              'out_features': n_autopology_features}},
                      {'name': 'Tanh', 'param': {}}


            ],
    "readout_hidden_nodes": [40, 20], # the number of nodes in each of the hidden layers 
                                      # of the readout network. The readout network acts on
                                      # the features after they have been convolved.
    # types of classical priors to use
    "bond_terms": ["morse"],
    # keys we're outputting 
    "output_keys": ["energy"],
    # whether we want the AuTopology weights to be learned
    "trainable_prior": True
}

In [20]:
from nff.train import get_model
CFF = get_model(autopology_params, model_type='AuTopology')

In [21]:
train_loader = DataLoader(dataset, batch_size=2, collate_fn=collate_dicts)

In [22]:
from nff.utils import batch_to
batch = batch_to( next(iter(train_loader)), "cpu")

In [23]:
# compute energy gradient 
CFF(batch)

{'energy': tensor([[0.0149],
         [0.0149]], grad_fn=<AddBackward0>),
 'energy_grad': tensor([[-1.7119e-04,  5.1298e-05,  1.9332e-04],
         [ 1.7119e-04, -5.1298e-05, -1.9332e-04],
         [-7.5648e-05,  8.1856e-06,  2.5942e-04],
         [ 7.5648e-05, -8.1856e-06, -2.5942e-04],
         [ 2.7062e-04,  4.4811e-05, -4.3333e-06],
         [-2.7062e-04, -4.4811e-05,  4.3333e-06],
         [-1.6878e-04,  1.6918e-04, -1.2313e-04],
         [ 1.6878e-04, -1.6918e-04,  1.2313e-04],
         [ 3.3862e-05, -2.5343e-04, -9.9191e-05],
         [-3.3862e-05,  2.5343e-04,  9.9191e-05],
         [-1.2438e-04, -2.2233e-04, -6.9771e-05],
         [ 1.2438e-04,  2.2233e-04,  6.9771e-05],
         [ 3.6733e-05, -1.4248e-04, -2.2912e-04],
         [-3.6733e-05,  1.4248e-04,  2.2912e-04],
         [ 1.7772e-05,  7.5201e-05, -2.5178e-04],
         [-1.7772e-05, -7.5201e-05,  2.5178e-04],
         [-1.5384e-05, -1.2671e-04, -2.3299e-04],
         [ 1.5384e-05,  1.2671e-04,  2.3299e-04],
         [ 